# Overview Materi

Jelaskan perbedaan singkat antara grid, randomized, bayesian search cv dengan optuna menurut pemahamanmu

source: https://www.youtube.com/watch?v=t-INgABWULw

# Import Data & Libraries

In [1]:
# jalankan hanya sekali
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 9.2 MB/s eta 0:00:00


In [20]:
# import library yang dibutuhkan di sini
import seaborn as sns
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt

In [4]:
df = sns.load_dataset('iris')
df.head()

,sepal_length,sepal_width,petal_length,petal_width,species
0,5.1,3.5,1.4,0.2,setosa
1,4.9,3.0,1.4,0.2,setosa
2,4.7,3.2,1.3,0.2,setosa
3,4.6,3.1,1.5,0.2,setosa
4,5.0,3.6,1.4,0.2,setosa


# Data Preprocessing

In [6]:
# ubah variabel kategorik ke numerik
df_encoded = pd.get_dummies(df, columns=["species"])
print(df_encoded.head())

   sepal_length  sepal_width  petal_length  petal_width  species_setosa  \
0           5.1          3.5           1.4          0.2            True   
1           4.9          3.0           1.4          0.2            True   
2           4.7          3.2           1.3          0.2            True   
3           4.6          3.1           1.5          0.2            True   
4           5.0          3.6           1.4          0.2            True   

   species_versicolor  species_virginica  
0               False              False  
1               False              False  
2               False              False  
3               False              False  
4               False              False  


In [5]:
# subsetting peubah
X = df.drop(['species'], axis=1)
y = df['species']

# Dataset Splitting

In [8]:
# split dengan rasio 80:20
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=21)

# Base Model Random Forest

In [16]:
# gunakan random forest classifier
rfr = RandomForestClassifier(random_state=42)
rfr.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [17]:
y_pred = rfr.predict(X_test)

In [21]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.3f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.3f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.933
Precision: 0.948
Recall: 0.933
F1 Score: 0.934
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        11
  versicolor       1.00      0.83      0.91        12
   virginica       0.78      1.00      0.88         7

    accuracy                           0.93        30
   macro avg       0.93      0.94      0.93        30
weighted avg       0.95      0.93      0.93        30



# Optuna

In [27]:
def objective(trial):
    n_estimators = trial.suggest_int("n_estimators", 50, 300)
    max_depth = trial.suggest_int("max_depth", 2, 20)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 4)

    model = RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        min_samples_split=min_samples_split,
        min_samples_leaf=min_samples_leaf,
        random_state=42)

    score = cross_val_score(model, X_train, y_train, cv=3, scoring="accuracy")

    return score.mean()

Hyperparameter dapat disesuaikan dengan algoritma yang digunakan. Kali ini kita menggunakan Random Forest sehingga yang dapat kita select adalah *n_estimators, max_depth, min_samples_split,* dan *min_samples_leaf*

In [28]:
study = optuna.create_study(direction='maximize', sampler=optuna.samplers.RandomSampler(seed=42))

[I 2025-09-30 13:52:58,389] A new study created in memory with name: no-name-f4c398f5-fd3e-4cab-b78d-120369722553


In [29]:
study.optimize(objective, n_trials=100)

[I 2025-09-30 13:53:03,785] Trial 0 finished with value: 0.975 and parameters: {'n_estimators': 144, 'max_depth': 20, 'min_samples_split': 8, 'min_samples_leaf': 3}. Best is trial 0 with value: 0.975.
[I 2025-09-30 13:53:04,184] Trial 1 finished with value: 0.975 and parameters: {'n_estimators': 89, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.975.
[I 2025-09-30 13:53:05,265] Trial 2 finished with value: 0.9666666666666667 and parameters: {'n_estimators': 200, 'max_depth': 15, 'min_samples_split': 2, 'min_samples_leaf': 4}. Best is trial 0 with value: 0.975.
[I 2025-09-30 13:53:07,079] Trial 3 finished with value: 0.975 and parameters: {'n_estimators': 258, 'max_depth': 6, 'min_samples_split': 3, 'min_samples_leaf': 1}. Best is trial 0 with value: 0.975.
[I 2025-09-30 13:53:07,674] Trial 4 finished with value: 0.9833333333333334 and parameters: {'n_estimators': 126, 'max_depth': 11, 'min_samples_split': 5, 'min_samples_leaf': 2}. Best is

it may take a while... so just wait n see ^^
<br>
they recommend to set n_trials at 100 cz it seems there's no significant score increase after 100 trials (also inefficient too, you'll have to wait in a quite long time)

In [33]:
study.best_params

{'n_estimators': 53,
 'max_depth': 4,
 'min_samples_split': 2,
 'min_samples_leaf': 1}

Berikut hasil hyperparameter tuning dari Optuna

In [35]:
# cek hasil hyperparameter tuning dari Optuna
best_params = study.best_params
print(f"Best Hyperparameters: {best_params}")

best_score = study.best_value
print(f"Best Accuracy: {best_score:.3f}")

Best Hyperparameters: {'n_estimators': 53, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 1}
Best Accuracy: 0.992


# Random Forest Using Optuna

In [36]:
# simpan hasil best hyperparameter tuning ke variabel bari
best_params = study.best_params
print("Best Params:", best_params)

Best Params: {'n_estimators': 53, 'max_depth': 4, 'min_samples_split': 2, 'min_samples_leaf': 1}


In [37]:
best_model = RandomForestClassifier(**best_params, random_state=42)

best_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=4, n_estimators=53, random_state=42)

In [38]:
y_pred = best_model.predict(X_test)

In [39]:
print(f"Accuracy: {accuracy_score(y_test, y_pred):.3f}")
print(f"Precision: {precision_score(y_test, y_pred, average='weighted'):.3f}")
print(f"Recall: {recall_score(y_test, y_pred, average='weighted'):.3f}")
print(f"F1 Score: {f1_score(y_test, y_pred, average='weighted'):.3f}")
print(classification_report(y_test, y_pred))

Accuracy: 0.933
Precision: 0.948
Recall: 0.933
F1 Score: 0.934
              precision    recall  f1-score   support

      setosa       1.00      1.00      1.00        11
  versicolor       1.00      0.83      0.91        12
   virginica       0.78      1.00      0.88         7

    accuracy                           0.93        30
   macro avg       0.93      0.94      0.93        30
weighted avg       0.95      0.93      0.93        30



Tidak terdapat kenaikan skor dengan sebelum menggunakan Optuna sebab skor yang dihasilkan melalui base model saja sudah bagus